In [1]:

"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow2/spoken_language_understanding/atis/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install tensorflow-gpu==2.0.0-alpha0 

In [3]:
import tensorflow as tf
import pprint
import logging
import time
import numpy as np

from sklearn.metrics import classification_report

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 2.0.0-alpha0
GPU Enabled: True


In [0]:
def get_vocab(vocab_path):
  word2idx = {}
  with open(vocab_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip()
      word2idx[line] = i
  return word2idx

In [0]:
def data_generator(f_path, params):
  print('Reading', f_path)
  with open(f_path) as f:
    for line in f:
      line = line.rstrip()
      text, slot_intent = line.split('\t')
      words = text.split()[1:-1]
      slot_intent = slot_intent.split()
      slots, intent = slot_intent[1:-1], slot_intent[-1]
      assert len(words) == len(slots)
      
      words = [params['word2idx'].get(w, len(params['word2idx'])) for w in words]
      intent = params['intent2idx'].get(intent, len(params['intent2idx']))
      slots = [params['slot2idx'].get(s, len(params['slot2idx'])) for s in slots]
      
      yield (words, (intent, slots))

In [0]:
def dataset(is_training, params):
  _shapes = ([None], ((), [None]))
  _types = (tf.int32, (tf.int32, tf.int32))
  _pads = (0, (-1, 0))
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['num_samples'])
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [0]:
params = {
  'train_path': '../data/atis.train.w-intent.iob',
  'test_path': '../data/atis.test.w-intent.iob',
  'word_path': '../vocab/word.txt',
  'vocab_path': '../vocab/word.npy',
  'intent_path': '../vocab/intent.txt',
  'slot_path': '../vocab/slot.txt',
  'batch_size': 16,
  'num_samples': 4978,
  'rnn_units': 300,
  'dropout_rate': 0.2,
  'clip_norm': 5.0,
  'lr': 3e-4,
  'num_patience': 3,
}

In [0]:
class Model(tf.keras.Model):
  def __init__(self, params: dict):
    super().__init__()
    self.embedding = tf.Variable(np.load(params['vocab_path']),
                                 dtype=tf.float32,
                                 name='pretrained_embedding')
    self.input_dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.bidir_gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
        params['rnn_units'], return_sequences=True))
    self.out_linear_intent = tf.keras.layers.Dense(params['intent_size'], name='output_intent')
    self.out_linear_slot = tf.keras.layers.Dense(params['slot_size'], name='output_slot')
  
  
  def call(self, inputs, training=False):
    if inputs.dtype != tf.int32:
      inputs = tf.cast(inputs, tf.int32)
    mask = tf.sign(inputs)
    self.input_dropout.noise_shape = (tf.shape(inputs)[0], 1, 300)
    
    x = tf.nn.embedding_lookup(self.embedding, inputs)
    x = self.input_dropout(x, training=training)
    x = self.bidir_gru(x, mask=None)
    
    x_intent = tf.reduce_max(x, 1)
    x_intent = self.out_linear_intent(x_intent)
    x_slot = self.out_linear_slot(x)
    return (x_intent, x_slot)

In [0]:
params['word2idx'] = get_vocab(params['word_path'])
params['intent2idx'] = get_vocab(params['intent_path'])
params['slot2idx'] = get_vocab(params['slot_path'])

params['word_size'] = len(params['word2idx']) + 1
params['intent_size'] = len(params['intent2idx']) + 1
params['slot_size'] = len(params['slot2idx']) + 1

In [0]:
def is_descending(history: list):
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True    

In [11]:
model = Model(params)
model.build(input_shape=(None, None))
pprint.pprint([(v.name, v.shape) for v in model.trainable_variables])

decay_lr = tf.optimizers.schedules.ExponentialDecay(params['lr'], 1000, 0.9)
optim = tf.optimizers.Adam(params['lr'])
global_step = 0

history_acc = []
best_acc = .0

t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)

while True:
  # TRAINING
  for (words, (intent, slots)) in dataset(is_training=True, params=params):
    with tf.GradientTape() as tape:
      y_intent, y_slots = model(words, training=True)
      loss_intent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=intent, logits=y_intent)
      loss_intent = tf.reduce_mean(loss_intent)
      # weighted cross entropy (weight of 'O' is set to be small)
      loss_slots = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=slots, logits=y_slots)
      weights = tf.cast(tf.sign(slots), tf.float32)
      padding = tf.constant(1e-2, tf.float32, weights.shape)
      weights = tf.where(tf.equal(weights, 0.), padding, weights)
      loss_slots = tf.reduce_mean(loss_slots * weights)
      # joint loss
      loss = loss_intent + loss_slots
    
    optim.lr.assign(decay_lr(global_step))
    grads = tape.gradient(loss, model.trainable_variables)
    grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
    optim.apply_gradients(zip(grads, model.trainable_variables))

    if global_step % 50 == 0:
      logger.info("Step {} | Loss: {:.4f} | Loss_intent: {:.4f} | Loss_slots: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(
          global_step, loss.numpy().item(), loss_intent.numpy().item(), loss_slots.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
      t0 = time.time()
    global_step += 1
    
  # EVALUATION
  acc_intent = tf.keras.metrics.Accuracy()
  acc_slots = tf.keras.metrics.Accuracy()
  slot_true = []
  slot_pred = []

  for (words, (intent, slots)) in dataset(is_training=False, params=params):
    y_intent, y_slots = model(words, training=False)
    y_intent = tf.argmax(y_intent, -1)
    y_slots = tf.argmax(y_slots, -1)
    
    acc_intent.update_state(y_true=intent, y_pred=y_intent)
    acc_slots.update_state(y_true=slots, y_pred=y_slots, sample_weight=tf.sign(slots))
    
    slot_true += slots.numpy().flatten().tolist()
    slot_pred += y_slots.numpy().flatten().tolist()
    
  acc_intent = acc_intent.result().numpy()
  acc_slots = acc_slots.result().numpy()
  logger.info("Evaluation: Testing Accuracy of Intent: {:.3f}".format(acc_intent))
  logger.info("Evaluation: Testing Accuracy of Slots ('O' filtered): {:.3f}".format(acc_slots))
  logger.info('\n'+classification_report(y_true = slot_true,
                                         y_pred = slot_pred,
                                         labels = list(params['slot2idx'].values()),
                                         target_names = list(params['slot2idx'].keys()),
                                         digits=3))
  history_acc.append(acc_slots)
  
  if acc_slots > best_acc:
    best_acc = acc_slots
    # you can save model here
  logger.info("Best Slot Accuracy ('O' filtered): {:.3f}".format(best_acc))
  
  if len(history_acc) > params['num_patience'] and is_descending(history_acc):
    logger.info("Testing Slot Accuracy not improved over {} epochs, Early Stop".format(params['num_patience']))
    break

W0418 07:34:18.427612 139750395750272 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:410: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


[('bidirectional/forward_unified_gru/kernel:0', TensorShape([300, 900])),
 ('bidirectional/forward_unified_gru/recurrent_kernel:0',
  TensorShape([300, 900])),
 ('bidirectional/forward_unified_gru/bias:0', TensorShape([2, 900])),
 ('bidirectional/backward_unified_gru/kernel:0', TensorShape([300, 900])),
 ('bidirectional/backward_unified_gru/recurrent_kernel:0',
  TensorShape([300, 900])),
 ('bidirectional/backward_unified_gru/bias:0', TensorShape([2, 900])),
 ('output_intent/kernel:0', TensorShape([600, 23])),
 ('output_intent/bias:0', TensorShape([23])),
 ('output_slot/kernel:0', TensorShape([600, 122])),
 ('output_slot/bias:0', TensorShape([122])),
 ('pretrained_embedding:0', TensorShape([750, 300]))]
Reading ../data/atis.train.w-intent.iob


I0418 07:34:20.791420 139750395750272 interactiveshell.py:2882] Step 0 | Loss: 3.6860 | Loss_intent: 2.7497 | Loss_slots: 0.9362 | Spent: 2.4 secs | LR: 0.000300
I0418 07:34:22.307185 139750395750272 interactiveshell.py:2882] Step 50 | Loss: 1.0111 | Loss_intent: 0.2928 | Loss_slots: 0.7184 | Spent: 1.5 secs | LR: 0.000298
I0418 07:34:23.829153 139750395750272 interactiveshell.py:2882] Step 100 | Loss: 1.0874 | Loss_intent: 0.6582 | Loss_slots: 0.4293 | Spent: 1.5 secs | LR: 0.000297
I0418 07:34:25.361568 139750395750272 interactiveshell.py:2882] Step 150 | Loss: 0.5909 | Loss_intent: 0.2295 | Loss_slots: 0.3614 | Spent: 1.5 secs | LR: 0.000295
I0418 07:34:26.891755 139750395750272 interactiveshell.py:2882] Step 200 | Loss: 0.5006 | Loss_intent: 0.1055 | Loss_slots: 0.3951 | Spent: 1.5 secs | LR: 0.000294
I0418 07:34:28.420959 139750395750272 interactiveshell.py:2882] Step 250 | Loss: 0.4756 | Loss_intent: 0.2742 | Loss_slots: 0.2015 | Spent: 1.5 secs | LR: 0.000292
I0418 07:34:29.9569

Reading ../data/atis.test.w-intent.iob


I0418 07:34:31.057031 139750395750272 interactiveshell.py:2882] Evaluation: Testing Accuracy of Intent: 0.888
I0418 07:34:31.057972 139750395750272 interactiveshell.py:2882] Evaluation: Testing Accuracy of Slots ('O' filtered): 0.789
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
I0418 07:34:31.082022 139750395750272 interactiveshell.py:2882] 
                              precision    recall  f1-score   support

                           O      0.989     0.580     0.731     11404
           B-toloc.city_name      0.570     0.986     0.722       716
         B-fro

...

I0418 07:39:36.595597 139750395750272 interactiveshell.py:2882] Evaluation: Testing Accuracy of Intent: 0.973
I0418 07:39:36.596653 139750395750272 interactiveshell.py:2882] Evaluation: Testing Accuracy of Slots ('O' filtered): 0.951
I0418 07:39:36.618615 139750395750272 interactiveshell.py:2882] 
                              precision    recall  f1-score   support

                           O      0.999     0.994     0.997     11404
           B-toloc.city_name      0.977     0.992     0.984       716
         B-fromloc.city_name      0.985     0.999     0.992       704
           I-toloc.city_name      0.960     0.985     0.972       265
      B-depart_date.day_name      0.991     0.991     0.991       212
              B-airline_name      1.000     1.000     1.000       101
         I-fromloc.city_name      0.972     0.994     0.983       177
 B-depart_time.period_of_day      1.000     0.923     0.960       130
              I-airline_name      0.956     1.000     0.977        65


Reading ../data/atis.train.w-intent.iob


I0418 07:39:37.734350 139750395750272 interactiveshell.py:2882] Step 8750 | Loss: 0.0007 | Loss_intent: 0.0001 | Loss_slots: 0.0006 | Spent: 3.0 secs | LR: 0.000119
I0418 07:39:39.277159 139750395750272 interactiveshell.py:2882] Step 8800 | Loss: 0.0005 | Loss_intent: 0.0000 | Loss_slots: 0.0005 | Spent: 1.5 secs | LR: 0.000119
I0418 07:39:40.808811 139750395750272 interactiveshell.py:2882] Step 8850 | Loss: 0.0011 | Loss_intent: 0.0001 | Loss_slots: 0.0010 | Spent: 1.5 secs | LR: 0.000118
I0418 07:39:42.337460 139750395750272 interactiveshell.py:2882] Step 8900 | Loss: 0.0042 | Loss_intent: 0.0008 | Loss_slots: 0.0035 | Spent: 1.5 secs | LR: 0.000117
I0418 07:39:43.881855 139750395750272 interactiveshell.py:2882] Step 8950 | Loss: 0.0012 | Loss_intent: 0.0007 | Loss_slots: 0.0005 | Spent: 1.5 secs | LR: 0.000117
I0418 07:39:45.432083 139750395750272 interactiveshell.py:2882] Step 9000 | Loss: 0.0026 | Loss_intent: 0.0008 | Loss_slots: 0.0018 | Spent: 1.5 secs | LR: 0.000116


Reading ../data/atis.test.w-intent.iob


I0418 07:39:47.664458 139750395750272 interactiveshell.py:2882] Evaluation: Testing Accuracy of Intent: 0.974
I0418 07:39:47.665412 139750395750272 interactiveshell.py:2882] Evaluation: Testing Accuracy of Slots ('O' filtered): 0.950
I0418 07:39:47.688871 139750395750272 interactiveshell.py:2882] 
                              precision    recall  f1-score   support

                           O      1.000     0.993     0.996     11404
           B-toloc.city_name      0.970     0.993     0.981       716
         B-fromloc.city_name      0.979     0.999     0.989       704
           I-toloc.city_name      0.953     0.985     0.968       265
      B-depart_date.day_name      0.991     0.991     0.991       212
              B-airline_name      1.000     1.000     1.000       101
         I-fromloc.city_name      0.972     0.994     0.983       177
 B-depart_time.period_of_day      1.000     0.923     0.960       130
              I-airline_name      0.942     1.000     0.970        65
